In [ ]:
import numpy as np
import pandas as pd

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score, recall_score
from sklearn.metrics import accuracy_score

rides = pd.read_csv("cab_rides.csv",delimiter=',')
weather = pd.read_csv("weather.csv",delimiter=',')

In [2]:
# print(rides)
# print(weather)

In [3]:
# Convert the timestamp into the desirable format
rides['date_time'] = pd.to_datetime(round(rides['time_stamp']/1000,0), unit='s')
weather['date_time'] = pd.to_datetime(weather['time_stamp'], unit='s')
rides.head()




,distance,cab_type,time_stamp,destination,source,price,surge_multiplier,id,product_id,name,date_time
0,0.44,Lyft,1544952607890,North Station,Haymarket Square,5.0,1.0,424553bb-7174-41ea-aeb4-fe06d4f4b9d7,lyft_line,Shared,2018-12-16 09:30:08
1,0.44,Lyft,1543284023677,North Station,Haymarket Square,11.0,1.0,4bd23055-6827-41c6-b23b-3c491f24e74d,lyft_premier,Lux,2018-11-27 02:00:24
2,0.44,Lyft,1543366822198,North Station,Haymarket Square,7.0,1.0,981a3613-77af-4620-a42a-0c0866077d1e,lyft,Lyft,2018-11-28 01:00:22
3,0.44,Lyft,1543553582749,North Station,Haymarket Square,26.0,1.0,c2d88af2-d278-4bfd-a8d0-29ca77cc5512,lyft_luxsuv,Lux Black XL,2018-11-30 04:53:03
4,0.44,Lyft,1543463360223,North Station,Haymarket Square,9.0,1.0,e0126e1f-8ca9-4f2e-82b3-50505a09db9a,lyft_plus,Lyft XL,2018-11-29 03:49:20


In [4]:
# weather.head()
# print(weather)

In [5]:
# # final_dataframe = rides.join(weather, on=['date_time'],rsuffix = '_w')
# final_dataframe = rides.merge(weather, on=['date_time'])

# #drop the null values rows

# final_dataframe=final_dataframe.dropna(axis=0)

# #make different columns of day and hour to simplify the format of date
# final_dataframe['day'] = final_dataframe.date_time.dt.dayofweek
# final_dataframe['hour'] = final_dataframe.date_time.dt.hour

In [6]:
# print(final_dataframe)

In [7]:
#make a column of merge date containing date merged with the location so that we can join the two dataframes on the basis of 'merge_date'
rides['merge_date'] = rides.source.astype(str) +" - "+ rides.date_time.dt.date.astype("str") +" - "+ rides.date_time.dt.hour.astype("str")
weather['merge_date'] = weather.location.astype(str) +" - "+ weather.date_time.dt.date.astype("str") +" - "+ weather.date_time.dt.hour.astype("str")



In [8]:
# final_dataframe = rides.join(weather, on=['merge_date'],rsuffix = '_w')
final_dataframe = rides.merge(weather, on=['merge_date'])

#drop the null values rows

final_dataframe=final_dataframe.dropna(axis=0)

#make different columns of day and hour to simplify the format of date
final_dataframe['day'] = final_dataframe.date_time_y.dt.dayofweek
final_dataframe['hour'] = final_dataframe.date_time_y.dt.hour

In [9]:
# change the index to merge_date column so joining the two datasets will not generate any error.

weather.index = weather['merge_date']

# we ignored surge value of more than 3 because the samples are very limited for surge_multiplier>3
surge_dataframe = final_dataframe[final_dataframe.surge_multiplier < 3]

In [10]:
# print(surge_dataframe)

In [11]:
# # filter to remove surge_multiplier of 1
# surge_dataframe.drop(surge_dataframe.index[surge_dataframe['surge_multiplier'] == 1])

# feature selection--> we are selecting the most relevant features from the dataset

x = surge_dataframe[['distance','day','hour','temp','clouds','pressure','humidity','wind','rain']]

y = surge_dataframe['surge_multiplier']

In [12]:
print(y)
print(np.max(y))
print(type(y))

192        1.0
193        1.0
194        1.0
195        1.0
196        1.0
          ... 
1263788    1.0
1263789    1.0
1263790    1.0
1263791    1.0
1263792    1.0
Name: surge_multiplier, Length: 190305, dtype: float64
2.5
<class 'pandas.core.series.Series'>


In [13]:
# print(y)

In [14]:
le = LabelEncoder()

#ignoring multiplier of 3 as there are only 2 values in our dataset
# le.fit([1.25,1.5,1.75,2.,2.25,2.5])
le.fit([1,1.25,1.5,1.75,2.,2.25,2.5])
y = le.transform(y)

feature_list=list(x.columns)
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.3,random_state=42) 

In [15]:
# print(y)

In [16]:
# Before Synthetic Minority Over-sampling TEchnique (SMOTE)
unique, counts = np.unique(y_train, return_counts=True)
print(dict(zip(unique, counts)))

{0: 129161, 1: 2181, 2: 951, 3: 482, 4: 414, 6: 24}


In [17]:
# After SMOTE
from imblearn.over_sampling import SMOTE
sm = SMOTE(random_state=42)
train_features, train_labels = sm.fit_resample(x_train, y_train)

test_features, test_labels = sm.fit_resample(x_test, y_test)

In [18]:
# print(train_labels)

In [19]:
# Model training
model = RandomForestClassifier(n_jobs=-1, random_state = 42, class_weight="balanced")

model.fit(x_train,y_train)
y_pred=model.predict(x_test)

In [20]:
# Feature importance
# Get numerical feature importances
importances = list(model.feature_importances_)
# List of tuples with variable and importance
feature_importances = [(feature, round(importance, 2)) for feature, importance in zip(feature_list, importances)]
# Sort the feature importances by most important first
feature_importances = sorted(feature_importances, key = lambda x: x[1], reverse = True)
# Print out the feature and importances
[print('Variable: {:20} Importance: {}'.format(*pair)) for pair in feature_importances]

Variable: distance             Importance: 0.46
Variable: temp                 Importance: 0.11
Variable: wind                 Importance: 0.11
Variable: pressure             Importance: 0.1
Variable: rain                 Importance: 0.09
Variable: humidity             Importance: 0.05
Variable: hour                 Importance: 0.04
Variable: clouds               Importance: 0.02
Variable: day                  Importance: 0.01


[None, None, None, None, None, None, None, None, None]

In [21]:
# Evaluation of the built model
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
f1_score(y_test, y_pred, average='weighted')
accuracy = accuracy_score(y_test, y_pred)
print('Model accuracy is ', round(accuracy * 100, 2), '%')

Model accuracy is  95.44 %


In [22]:
# print(np.max(y_test))

In [23]:
# print(np.sum(y_pred))